In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [22]:
train = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet")

val  = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet')

test = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet')

In [23]:
train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2022-01-01 00:14:21,2022-01-01 00:15:33,N,1.0,42,42,1.0,0.44,3.5,0.50,0.5,0.00,0.0,None,0.3,4.80,2.0,1.0,0.00
1,1,2022-01-01 00:20:55,2022-01-01 00:29:38,N,1.0,116,41,1.0,2.10,9.5,0.50,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.00
2,1,2022-01-01 00:57:02,2022-01-01 01:13:14,N,1.0,41,140,1.0,3.70,14.5,3.25,0.5,4.60,0.0,None,0.3,23.15,1.0,1.0,2.75
3,2,2022-01-01 00:07:42,2022-01-01 00:15:57,N,1.0,181,181,1.0,1.69,8.0,0.50,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2022-01-01 00:07:50,2022-01-01 00:28:52,N,1.0,33,170,1.0,6.26,22.0,0.50,0.5,5.21,0.0,None,0.3,31.26,1.0,1.0,2.75


In [24]:
dv = DictVectorizer()

In [25]:
def read_dataframe(df: pd.DataFrame):
    #df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [26]:
def data_processing(train_df, val_df, test_df):
    df_train = read_dataframe(train_df)
    df_val = read_dataframe(val_df)
    df_test = read_dataframe(test_df)

    target = 'tip_amount'

    y_train = df_train[target].values
    y_val= df_val[target].values
    y_test = df_test[target].values

    """Init the dictvectorizer""" 
    dv = DictVectorizer()
    X_train, dv = preprocess(df_train, dv, fit_dv=True)
    X_val, _ = preprocess(df_val, dv, fit_dv=False)
    X_test, _ = preprocess(df_test, dv, fit_dv=False)

    return X_train,y_train,X_val,y_val,X_test,y_test,dv

    

In [29]:
X_train,y_train,X_val,y_val,X_test,y_test,dv = data_processing(train, val, test)

/tmp/ipykernel_2791/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)
/tmp/ipykernel_2791/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)
/tmp/ipykernel_2791/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [9]:
#save the dictvectorizer
import pickle
with open('2_homework/dv.pkl', 'wb') as file:
    pickle.dump(dv, file)

answer to the Q2

In [10]:
#To check the size of the file
!du -h "/home/ubuntu/notebooks/2_homework/dv.pkl"

#the answer is b-154kb

152K	/home/ubuntu/notebooks/2_homework/dv.pkl


------------------------------------------------

Q3 Train a model with autolog

In [10]:
def train_func(X_train,y_train,X_val,y_val,model):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_pred,y_val)
    return rmse


In [13]:
model  = RandomForestRegressor(max_depth=10, random_state=0)

In [15]:
mlflow.set_tracking_uri(uri="http://localhost:5000")
mlflow.set_experiment("2_homework")



<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>

In [36]:
with mlflow.start_run():
    mlflow.set_tag('Model','RandomForestRegressor')
    mlflow.sklearn.autolog()

    #run the model
    rmse = train_func(X_train,y_train,X_val,y_val,model)

    #run the metric logging
    mlflow.log_metric('RMSE',rmse)

    #log the model artifact
    mlflow.sklearn.log_model(model, artifact_path='models')

2024/02/02 06:46:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


In [16]:
# now we have been tasked to check the max_depth of the model
# we will try it using mlflow client

from mlflow.client import MlflowClient

client = MlflowClient(tracking_uri=mlflow.get_tracking_uri())

In [17]:
client.search_experiments()
#This lists all the experiments

[<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1706710158792, experiment_id='4', last_update_time=1706710158792, lifecycle_stage='active', name='lightgbm prediction experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1706708938289, experiment_id='3', last_update_time=1706708938289, lifecycle_stage='active', name='2_model_registry', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/2', creation_time=1705592485729, experiment_id='2', last_update_time=1705592485729, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/1', creation_time=1705591891060, experiment_id='1', last_update_time=1705591891060, lifecycle_stage='active', name='nyc-taxi-exper

This answers the 3rd question

In [18]:
runs = client.search_runs(
    experiment_ids=5
)


#since we know our experiment id is 5

for run in runs:
    print(f"the max_depth is {run.data.params['max_depth']}")





the max_depth is 10


===================================================================

# Q4

In [ ]:
#%pip install optuna
#first  lets define the objective function to try different values to tune using optuna 
# then define the objective function


#import the necessary libs



In [47]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

  #init the mode

In [19]:



def objective(trial):
    #define the hyperparams
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 50, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 20, 1),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10, 1),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4, 1),
        'random_state': 42,
        'n_jobs': -1
    }
    #model and features and targets have been initialized following previous experiments
    model = RandomForestRegressor(**params)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    return rmse



In [ ]:
#Conduct the study and store the params in the study object
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective,n_trials=10)

In [45]:
trials=study.get_trials() #get all the trials conducted


[FrozenTrial(number=0, state=1, values=[2.4539999227851617], datetime_start=datetime.datetime(2024, 2, 2, 8, 43, 47, 535833), datetime_complete=datetime.datetime(2024, 2, 2, 8, 43, 52, 195588), params={'n_estimators': 41, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=50, log=False, low=10, step=1), 'max_depth': IntDistribution(high=20, log=False, low=1, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=4, log=False, low=1, step=1)}, trial_id=0, value=None),
 FrozenTrial(number=1, state=1, values=[2.4526464641463614], datetime_start=datetime.datetime(2024, 2, 2, 8, 43, 52, 196689), datetime_complete=datetime.datetime(2024, 2, 2, 8, 43, 52, 987816), params={'n_estimators': 16, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2}, user_attrs={}, system_attrs={}, intermediate_values

In [46]:
trials[0].number

0

 Now for each trial conduct an exp


In [ ]:
#before that we will need to delete the existing deleted experiment permanently

In [52]:

#create an experiment
mlflow.set_experiment("random-forest-hyperopt")
for trial in trials:
    with mlflow.start_run():
        mlflow.set_tag("Trial No",trial.number)

        #logs
        mlflow.log_params(trial.params)

        model = RandomForestRegressor(**trial.params)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val,y_pred, squared=False)
        #log the artifact
        mlflow.sklearn.log_model(model,artifact_path="models")

        #log the metrics
        mlflow.log_metric('rmse',rmse)


MlflowException: Cannot set a deleted experiment 'random-forest-hyperopt' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [ ]:

        #get the best model
best_model = RandomForestRegressor(**study.best_params)

        #log the best model
mlflow.sklearn.log_model(best_model,artifact_path="models")

        #eval the model
best_model.fit(X_train,y_train)
        y_pred = best_model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse',rmse)

    

NameError: name 'study' is not defined

In [3]:
from mlflow.client import MlflowClient
client = MlflowClient(tracking_uri="http://localhost:5000")


In [4]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/6', creation_time=1706861480031, experiment_id='6', last_update_time=1706861480031, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1706710158792, experiment_id='4', last_update_time=1706710158792, lifecycle_stage='active', name='lightgbm prediction experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1706708938289, experiment_id='3', last_update_time=1706708938289, lifecycle_stage='active', name='2_model_registry', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/2', creation_time=1705592485729, experiment_id='2', last_update_time=1705592485729, lifecycle_stage='active', name='nyc_taxi_experiment', t

In [10]:
runs = client.search_runs(experiment_ids=6,max_results=1)
for run in runs:
    print(f"The best rmse score is {run.data.metrics}")

The best rmse score is {'rmse': 2.4481454701058243}


---------------------------------------------------------

# Q5

In [12]:
##REGISTER THE BEST MODEL

from mlflow.client import MlflowClient
from mlflow.entities import ViewType



mlflow.set_experiment('random_forest_best_model')
#mlflow.set_tracking_uri(uri='http://localhost:5000')

2024/02/02 08:29:45 INFO mlflow.tracking.fluent: Experiment with name 'random_forest_best_model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7', creation_time=1706862585292, experiment_id='7', last_update_time=1706862585292, lifecycle_stage='active', name='random_forest_best_model', tags={}>

In [ ]:
with mlflow.start_run():
    